In [1]:
from glob import glob

import pandas as pd
import polars as pl


In [2]:
csv_parts = glob('data/part-00*.parquet')
df = pl.concat(map(pl.read_parquet, csv_parts))

In [3]:
df = df.filter(pl.col('url_host_tld') != "")

In [4]:
(df.unique(pl.col('url_host_tld'), keep='first', maintain_order=True)
 .select(pl.col('url_host_tld'))
 )

url_host_tld
str
"""com"""


In [5]:
df_prepared=df.unique(pl.col('url_host_registered_domain'), keep='first', maintain_order=True)
df_prepared.select(pl.col('url_host_registered_domain')).head(50)

url_host_registered_domain
str
"""ldkeyfinder.com"""
"""ldkh.com"""
"""ldkhr.com"""
"""ldkid.com"""
"""ldkirklin.com"""
…
"""ldlc-pro.com"""
"""ldlcarena.com"""
"""ldlcasvelfeminin.com"""


In [6]:
df_prepared.select(pl.col('url_host_registered_domain').str.starts_with('www.')).filter(pl.col('url_host_registered_domain') == True)

url_host_registered_domain
bool


In [7]:
df_prepared.select(pl.col('url_host_registered_domain').str.starts_with('http')).filter(pl.col('url_host_registered_domain') == True)

url_host_registered_domain
bool


In [8]:
df_prepared=df_prepared.with_columns(
 ("https://"+pl.col('url_host_registered_domain')).alias('url_host_registered_domain')
)

In [9]:
df_prepared.write_csv('data/com_domains.csv')